### Applications of Data Science
# Lab 3
# Preparing Text Data with R

## Overview

This lab introduces you to the basics of text mining and text data preparation. In this lab you will work with a set of 160,000 tweets, which include sentiment labels. 

Social media sentiment is an important indicator of public opinion.  Determining sentiment can be valuable in a number of applications including brand awareness, product launches, and detecting political trends. 

Raw text is inherently messy. Machine understanding and analysis is inhibited by the presence of extraneous symbols and words that clutter the text. The exact nature of the required text cleaning depends on the application.  In this case, you will focus on text cleaning to facilitate sentiment classification.The presence of certain words determines the sentiment of the tweet. Words and symbols which are extraneous to this purpose are distractions at best, and a likely source of noise in the analysis. You will follow these steps to prepare the tweet text for analysis: 

- Remove punctuation symbols and numerals, leaving only alphabetic characters.
- Convert all remaining characters to lower case. 
- Remove stopwords like "the", "and" and "this". Since these words are relatively common, yet communicate no particular sentiment, they can bias analytics. 
- Stem all remaining words to their root stem. 


## What you will need
To complete this lab, you will need the following:
- A web browser and Internet connection
- An Azure ML workspace
- The lab files for this lab


## Load and transform the tweet data

As a first step, ensure that you have uploaded the **tweets.csv** and **stopwords.csv ** files as new datasets in your Azure Machine Learning workspace. Then use the following code to load the tweets data set and set the column names to convenient values.

In [ ]:
library(tm)
library("AzureML")
ws <- workspace()
dataset <- download.datasets(ws, "tweets.csv")
colnames(dataset) <- c("sentiment", "tweets") # Set the column names
head(dataset)

Examine the head of the data frame shown above, noticing the content of the two columns.

- The **Sentiment** column contains a sentiment score ``{0,4}`` for negative of positive sentiment of the tweet.
- The **Tweets** column contains the actual text of the tweet. 

In order to work with the text in the tweets using the tools in the R tm package, you must convert them to a corpus object. A tm vector corpus is a vector of corpus objects. In this case, the text of each tweet is a single corpus. 

Execute the code in the cell below to create a tm vector corpus object. 

In [ ]:
tweet.text <- Corpus(VectorSource(dataset['tweets']))
class(tweet.text)

## Clean and Lower Case Symbols with R 
In this exercise you will take the first steps in preparing the tweet text using custom R code. You will use the R Text Mining package **tm** to perform some basic filtering and cleaning of the tweet text. Using the tools in the tm package you will perform the following steps:

- Remove numbers.
- Remove punctuation.
- Remove excess white space.
- Convert to lower case.

Each line in the code cell below using the **tm_map** funciton to apply the specified transformation to the text. Execute the code in the cell below to clean and lower case the tweet text. 

In [ ]:
tweet.text <- tm_map(tweet.text, content_transformer(removeNumbers))
tweet.text <- tm_map(tweet.text, content_transformer(removePunctuation))
tweet.text <- tm_map(tweet.text, content_transformer(stripWhitespace))
tweet.text <- tm_map(tweet.text, content_transformer(tolower))

Next, you will create a term-document matrix (TDM) of the tweets. A TDM is a sparse matrix structure with the words (terms) in the rows and documents which may or may not contain that word in the columns. The count of word occurrence of the document is contained in the cells.

The frequency of words is simply computed by summing the values in the rows. Note that the row_sums function from the slam package is used to deal with the sparse matrix structure. A dataframe is then constructed with the words and their frequencies in descending order. 

Execute the code in the cell below to compute the TDM, the word frequencies and examine the head of the data frame. This may take a considerable amount of time to run:

In [ ]:
to.WF = function(corpus){
    require(tm)
    ## Compute a term-document matrix and then 
    ## compute the word frequencies.
    library(slam)
    tdm <- TermDocumentMatrix(corpus, control = list(stopwords = FALSE))
    tdm <- removeSparseTerms(tdm, 0.9999999)
    freq <- row_sums(tdm, na.rm = T)   
    ## Sort the word frequency and build a dataframe
    ## including the cumulative frequecy of the words.
    freq <- sort(freq, decreasing = TRUE)
    word.freq <- data.frame(word = factor(names(freq), levels = names(freq)), 
                        freq = freq)
    word.freq['Cum'] <- cumsum(word.freq['freq'])/sum(word.freq$freq)
    word.freq
}

wf = to.WF(tweet.text)
head(wf, n = 20)

Notice that the most frequent words are in the head of this data frame. Of these 20 most frequent words, only one, 'good', is likely to convey much information on sentiment.  

You can examine the normalized text in the processed tweets by executing the code in the cell below. 

In [ ]:
head(data.frame(sentiment = dataset$sentiment, 
                text = enc2utf8(unlist(sapply(tweet.text, `[`, "content"))), stringsAsFactors=F))

Examine this text. Notice that all text is lower case and there are no numbers, punctuation or special characters. 

Now examine the head of the resulting word frequency data frame to determine the following:

- What is the percentage of all words for these first 20 words? 
- Of these 20 words, how many are likely to contibute sentiment information? 
- Are these 20 words different from the words seen for the raw text? 

In [ ]:
head(wf, n = 20)

## Remove stop words

In the previous section you removed extraneous characters and whitespace from the tweet text. The results show that the most frequent words do not communicate much sentiment information. These frequent words, which are largely extraneous, are known as stop words and should be removed from the text before further analysis. In this exercise you will use custom R code to remove stop words from the tweet text.

As a first step you will load the list of stop words, and examine the first 100 by executing the code in the cell below (again, this may take some time to run):

In [ ]:
stop.words <- download.datasets(ws, "stopwords.csv")
stop.words = unique(stop.words)
stop.words[1:100,]

Examine the stop word list and notice the following:

- These words are generally common in English language text.
- None of these words seem likely to indicate any particular sentiment. 
- Some of these words, like 'aww', are specialized to this application of analyzing tweets. 

The code in the cell below applies the **removeWords** operation to the tweet text. Execute the code in the cell to remove the stop words from the tweets and plot the word frequency.

In [ ]:
tweet.text <- tm_map(tweet.text, removeWords, stop.words[, 'words'])
wf = to.WF(tweet.text)

You will now examine the head of the resulting word frequency data frame to determine the following:

- What is the percentage of all words for these first 20 words? 
- Of these 20 words, how many are likely to contribute sentiment information? 
- Are these 20 words different from the words seen for the normalized text? 

To perform this exercise, apply the **head** function, with the **n = 20** argument, to the **wf** data frame. 
****

In [ ]:
head(wf, n = 20)

## Stem the Words

You have cleaned the tweet text and removed stop words. There is one last data preparation step required, stemming the words. Stemming is a process of reducing words to their stems or roots. For example, conjugated verbs such as "goes", "going", and "gone" are stemmed to the word "go".  Both Python and R offer a choice of stemmers. Depending on this choice, the results can be more or less suitable for the application. In this case, you will use the popular Porter stemmer. 

The Porter stemmer is in the R **SnowBallC** library. Execute the code in the cell below to load and apply the Porter stemmer to the tweet text (again, this may take some time to run)

In [ ]:
library(SnowballC) ## For stemming words
tweet.text <- tm_map(tweet.text, stemDocument)
wf = to.WF(tweet.text)

Finally, examine a sample of the prepared tweet text: 

In [ ]:
head(data.frame(sentiment = dataset$sentiment, 
                text = enc2utf8(unlist(sapply(tweet.text, `[`, "content"))), stringsAsFactors=F))

You will now examine the head of the resulting word frequency data frame to determine the following:

- Have any of the words in the list been stemmed? 
- Has the stemming changed the frequency of these words?

In [ ]:
head(wf, n=20)